In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/ibm-granite/granite-3.3-2b-instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/ibm-granite/granite-3.3-2b-instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="ibm-granite/granite-3.3-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [1]:
"""
StudyMate — Gradio web application (single-file)
Created for hackathon use: attractive UI, PDF upload, semantic search (FAISS), RAG with a chat interface.

Features:
- Upload one or more PDFs
- Clean text extraction with PyMuPDF (fitz)
- Chunking with overlap
- Embeddings with SentenceTransformers
- FAISS vector store
- Chat UI (Gradio Blocks): conversation, context viewer, top source highlights, download chat transcript
- Two model modes:
    1) Local model (transformers AutoModelForCausalLM) — if you have enough RAM/GPU
    2) Hugging Face Inference API — lightweight (needs HF token) — recommended for hackathons/demo

How to run (recommended for quick hackathon demo):
1. Create a virtualenv or use Colab.
2. Install dependencies:
   pip install gradio==3.36.1 transformers sentence-transformers faiss-cpu pymupdf accelerate safetensors
   # If using HF Inference API: pip install requests
3. Export HF token if you plan to use the Inference API:
   export HUGGINGFACE_TOKEN="hf_xxx"  # or set on Windows via SET
4. Run:
   python StudyMate_Gradio_App.py

Notes for hackathon:
- Use HF Inference API to avoid long model loads and GPU requirements.
- For local LLM, pick a model you can run on your hardware (e.g., small Llama-family or IBM Granite if available).

"""

import os
import re
import tempfile
import time
import json
from pathlib import Path
from typing import List, Tuple

import fitz  # PyMuPDF
import numpy as np

import gradio as gr

# Embeddings
from sentence_transformers import SentenceTransformer

# Vector store
import faiss

# Optional local model (transformers) or HF Inference API
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM
    import torch
    TRANSFORMERS_AVAILABLE = True
except Exception:
    TRANSFORMERS_AVAILABLE = False

# Optional requests for HF Inference API
try:
    import requests
    REQUESTS_AVAILABLE = True
except Exception:
    REQUESTS_AVAILABLE = False

# ----------------------
# Utility functions
# ----------------------

def extract_text_from_pdf(file_path: str) -> str:
    """Extract plain text from a PDF file using PyMuPDF (fitz)."""
    doc = fitz.open(file_path)
    text = []
    for page in doc:
        text.append(page.get_text("text"))
    return "\n".join(text)


def clean_whitespace(text: str) -> str:
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def chunk_text(text: str, chunk_size: int = 800, overlap: int = 200) -> List[str]:
    """Split text into overlapping chunks (character-based)."""
    chunks = []
    start = 0
    L = len(text)
    while start < L:
        end = min(start + chunk_size, L)
        chunk = text[start:end]
        chunks.append(chunk.strip())
        start += chunk_size - overlap
        if start >= L:
            break
    return chunks


# ----------------------
# Embedding and FAISS
# ----------------------
class VectorStore:
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.embed_model = SentenceTransformer(model_name)
        self.index = None
        self.metadata = []  # list of dicts {"text":..., "source":...}
        self.dim = None

    def build(self, texts: List[str], metadatas: List[dict]):
        enc = self.embed_model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
        self.dim = enc.shape[1]
        self.index = faiss.IndexFlatL2(self.dim)
        self.index.add(enc)
        self.metadata = metadatas

    def add(self, texts: List[str], metadatas: List[dict]):
        enc = self.embed_model.encode(texts, convert_to_numpy=True, show_progress_bar=False)
        if self.index is None:
            self.dim = enc.shape[1]
            self.index = faiss.IndexFlatL2(self.dim)
        self.index.add(enc)
        self.metadata.extend(metadatas)

    def search(self, query: str, k: int = 4) -> List[Tuple[dict, float]]:
        q = self.embed_model.encode([query], convert_to_numpy=True)
        if self.index is None:
            return []
        distances, indices = self.index.search(q, k)
        results = []
        for idx, dist in zip(indices[0], distances[0]):
            if idx < 0 or idx >= len(self.metadata):
                continue
            results.append((self.metadata[idx], float(dist)))
        return results


# ----------------------
# LLM wrapper
# ----------------------
class LLM:
    def __init__(self, mode: str = "hf_api", hf_model: str = None, hf_token: str = None, local_model: str = None):
        """
        mode: 'hf_api' (Hugging Face Inference API) or 'local' (transformers)
        hf_model: model id for HF inference API
        local_model: model id for AutoModelForCausalLM
        """
        self.mode = mode
        self.hf_model = hf_model
        self.hf_token = hf_token or os.environ.get("HUGGINGFACE_TOKEN")
        self.local_model = local_model
        self._loaded = False
        self.tokenizer = None
        self.model = None
        if self.mode == "local" and not TRANSFORMERS_AVAILABLE:
            raise RuntimeError("transformers not available for local mode")

    def load_local(self):
        if self._loaded:
            return
        assert self.local_model is not None
        self.tokenizer = AutoTokenizer.from_pretrained(self.local_model)
        # load with low_cpu_mem_usage in recent transformers
        self.model = AutoModelForCausalLM.from_pretrained(self.local_model, device_map="auto")
        self._loaded = True

    def generate(self, prompt: str, max_tokens: int = 256) -> str:
        if self.mode == "hf_api":
            if not REQUESTS_AVAILABLE:
                return "ERROR: requests not installed. Install requests to use HF Inference API."
            assert self.hf_model is not None, "HF model id required for hf_api mode"
            assert self.hf_token is not None, "HUGGINGFACE_TOKEN missing for hf_api mode"
            api_url = f"https://api-inference.huggingface.co/models/{self.hf_model}"
            headers = {"Authorization": f"Bearer {self.hf_token}"}
            payload = {"inputs": prompt, "parameters": {"max_new_tokens": max_tokens}}
            resp = requests.post(api_url, headers=headers, json=payload)
            if resp.status_code != 200:
                return f"Inference API error {resp.status_code}: {resp.text}"
            out = resp.json()
            if isinstance(out, dict) and out.get("error"):
                return f"Inference API error: {out.get('error')}"
            # HF Inference API sometimes returns a list of dicts
            if isinstance(out, list) and len(out) > 0 and "generated_text" in out[0]:
                return out[0]["generated_text"]
            if isinstance(out, dict) and "generated_text" in out:
                return out["generated_text"]
            return str(out)
        else:
            # local generation
            if not self._loaded:
                self.load_local()
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            with torch.no_grad():
                outputs = self.model.generate(**inputs, max_new_tokens=max_tokens)
            decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            # remove prompt prefix
            return decoded[len(prompt):].strip()


# ----------------------
# Application state
# ----------------------
STATE = {
    "vectorstore": None,
    "llm": None,
    "chunks": [],
    "metadatas": [],
}

# ----------------------
# Core pipeline actions
# ----------------------

def ingest_pdfs(files, chunk_size=800, overlap=200):
    """Extract, chunk, and build vector store. Returns summary info."""
    all_texts = []
    metadatas = []
    for f in files:
        fname = f.name if hasattr(f, "name") else f
        # save to temp and extract
        if hasattr(f, "name"):
            tmp_path = f.name
        else:
            tmp_path = str(f)
        text = extract_text_from_pdf(tmp_path)
        text = clean_whitespace(text)
        chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)
        for i, c in enumerate(chunks):
            all_texts.append(c)
            metadatas.append({"source": os.path.basename(tmp_path), "chunk_id": i, "text": c[:500]})

    vs = VectorStore()
    vs.build(all_texts, metadatas)
    STATE["vectorstore"] = vs
    STATE["chunks"] = all_texts
    STATE["metadatas"] = metadatas
    return {"n_chunks": len(all_texts), "n_files": len(files)}


def answer_question(question: str, k: int = 4, llm_mode: str = "hf_api", hf_model: str = None, hf_token: str = None, local_model: str = None) -> Tuple[str, List[dict]]:
    """Search vectorstore, build context, and call LLM. Returns (answer, sources)."""
    vs: VectorStore = STATE.get("vectorstore")
    if vs is None:
        return "Please upload PDFs and build the knowledge base first.", []

    results = vs.search(question, k=k)
    # aggregate context
    context_pieces = []
    sources = []
    for meta, dist in results:
        context_pieces.append(meta["text"])
        sources.append({"source": meta["source"], "chunk_id": meta["chunk_id"], "dist": dist, "snippet": meta["text"]})

    context = "\n\n---\n\n".join(context_pieces)

    prompt = (
        "You are StudyMate, an AI academic assistant. Use the provided context from the user's uploaded PDFs to answer the question. If the answer is not contained in the context, say you don't know and provide suggestions to locate the answer."
        f"\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    )

    # initialize LLM wrapper if necessary
    if STATE.get("llm") is None or STATE["llm"].mode != llm_mode:
        STATE["llm"] = LLM(mode=llm_mode, hf_model=hf_model, hf_token=hf_token, local_model=local_model)

    llm = STATE["llm"]
    answer = llm.generate(prompt, max_tokens=256)
    return answer, sources


# ----------------------
# Gradio UI
# ----------------------

css = """
:root{
  --primary:#7c3aed; /* violet-600 */
  --accent:#06b6d4;  /* cyan-500 */
}
.gradio-container {background: linear-gradient(180deg, rgba(124,58,237,0.06), rgba(6,182,212,0.03));}
.header{padding:18px;border-radius:14px;margin-bottom:12px}
.card{background:white;padding:14px;border-radius:12px;box-shadow:0 6px 24px rgba(15,23,42,0.06)}
.small-muted{font-size:12px;color:#6b7280}
.source-bubble{background:rgba(15,23,42,0.04);padding:10px;border-radius:8px;margin:6px 0}
"""

with gr.Blocks(css=css, title="StudyMate — Hackathon-ready", theme=gr.themes.Soft()) as app:
    with gr.Row(elem_id="top-row"):
        with gr.Column(scale=1):
            gr.Markdown("# 🎓 StudyMate — Conversational PDF Q&A")
            gr.Markdown("Upload PDFs, then ask questions. Designed for hackathons: fast demo mode via Hugging Face Inference API or local model option.")
            with gr.Accordion("How it works", open=False):
                gr.Markdown(
                    "1. Upload PDFs → 2. We extract & chunk text → 3. Build embeddings & FAISS index → 4. Ask questions (RAG) using an LLM."
                )

        with gr.Column(scale=1):
            # Controls card
            with gr.Column(elem_classes="card header"):
                hf_token_input = gr.Textbox(label="Hugging Face Token (optional for HF Inference API)", placeholder="hf_xxx (recommended for quick demos)", type="password")
                hf_model_input = gr.Textbox(label="HF model (inference API) [default: 'gpt2']", value="gpt2")
                local_model_input = gr.Textbox(label="Local model ID (transformers) - optional", placeholder="e.g. ibm-granite/granite-3.3-2b-instruct")
                llm_mode_radio = gr.Radio(label="LLM Mode", choices=["hf_api", "local"], value="hf_api")
                chunk_size_slider = gr.Slider(label="Chunk size (chars)", minimum=300, maximum=2000, step=50, value=800)
                overlap_slider = gr.Slider(label="Chunk overlap (chars)", minimum=50, maximum=800, step=25, value=200)

    with gr.Row():
        with gr.Column(scale=1):
            upload = gr.File(label="Upload PDF(s)", file_count="multiple", file_types=[".pdf"])
            ingest_btn = gr.Button("Build Knowledge Base", elem_id="ingest_btn")
            ingest_output = gr.Textbox(label="Ingest status", interactive=False)

            gr.Markdown("---")
            gr.Markdown("### Conversation")
            chatbot = gr.Chatbot(label="StudyMate Chat", height=380)
            user_input = gr.Textbox(label="Your question", placeholder="Ask anything about the uploaded PDFs...")
            with gr.Row():
                ask_btn = gr.Button("Ask")
                clear_btn = gr.Button("Clear chat")

        with gr.Column(scale=0.7):
            gr.Markdown("### Top Sources")
            sources_box = gr.Markdown("Upload PDFs and build the KB to see top sources here.")
            gr.Markdown("---")
            gr.Markdown("### Settings & Export")
            k_slider = gr.Slider(label="# retrieved chunks (k)", minimum=1, maximum=10, step=1, value=4)
            download_btn = gr.Button("Download Transcript")

    # Hidden state
    transcript_store = gr.State([])

    def on_ingest(files, chunk_size, overlap, hf_token_field):
        if not files:
            return "No files uploaded." , None
        # gradio File gives temp filenames; just pass them
        files_list = files
        info = ingest_pdfs(files_list, chunk_size=int(chunk_size), overlap=int(overlap))
        msg = f"Indexed {info['n_chunks']} chunks from {info['n_files']} files. Ready to ask questions."
        if hf_token_field:
            os.environ["HUGGINGFACE_TOKEN"] = hf_token_field
        return msg, info

    ingest_btn.click(on_ingest, inputs=[upload, chunk_size_slider, overlap_slider, hf_token_input], outputs=[ingest_output, ingest_output])

    def on_ask(question, k, llm_mode, hf_model, hf_token, local_model, chat_history, transcript):
        if not question or question.strip()=="":
            return chat_history, transcript
        # call answer pipeline
        answer, sources = answer_question(question, k=int(k), llm_mode=llm_mode, hf_model=hf_model, hf_token=hf_token, local_model=local_model)
        # append to chat
        chat_history = chat_history + [("You", question), ("StudyMate", answer)]
        # update transcript
        transcript = transcript + [{"q":question, "a":answer, "sources":sources}]
        # update sources box text
        sources_md = "### Top Sources\n"
        for s in sources:
            sources_md += f"- **{s['source']}** (dist={s['dist']:.3f}) — {s['snippet'][:200]}...\n"
        return chat_history, transcript, sources_md

    ask_btn.click(on_ask, inputs=[user_input, k_slider, llm_mode_radio, hf_model_input, hf_token_input, local_model_input, chatbot, transcript_store], outputs=[chatbot, transcript_store, sources_box])

    def clear_chat():
        return [], []
    clear_btn.click(clear_chat, outputs=[chatbot, transcript_store])

    def download_transcript(transcript):
        if not transcript:
            return None
        fname = f"transcript_{int(time.time())}.json"
        with open(fname, "w", encoding="utf-8") as f:
            json.dump(transcript, f, ensure_ascii=False, indent=2)
        return gr.File.update(value=fname, visible=True)

    download_btn.click(download_transcript, inputs=[transcript_store], outputs=[gr.File()])

    # Run
    demo = app

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7860, share=True)


/tmp/ipython-input-2621103597.py:313: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="StudyMate Chat", height=380)
/usr/local/lib/python3.12/dist-packages/gradio/layouts/column.py:59: UserWarning: 'scale' value should be an integer. Using 0.7 will cause issues.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12df33bc52d9f6d2bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
